# Task 3 – Construction the network

After successfully creating both the custom dataset and the dataloader, you need to create a
neural network, and use the data loader to feed the network. The architecture, complexity and
regularization are all up to you, but you need to justify your choices in comments. You are
more than welcome to replicate already known architectures or architectures we made during
the course, but you are NOT allowed to use any pretrained networks. You are also not
allowed to use any training data that is not included on ItsLearning.

Carefully consider which hyperparameters to test and strategically try to find the optimal
architecture for the task. In the comments, please describe your method for the optimization
and your choice of hyperparameters. Remember that there is an underlying competition, and
the highest accuracy wins. The competition will be measured based on the saved model, so
make sure to submit only the best one!

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import glob
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using CUDA


In [ ]:
rgb = 3
greyscale = 1
h, w = 256,256

In [ ]:
input_dim=(h,w)
channel_dim=greyscale

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using CUDA")
else:
    device = torch.device("cpu")
    print("Using CPU")

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_size, class_names, path=None, transformations=None, num_per_class: int = -1, channel_dim='rgb'):
        self.img_size = img_size
        self.path = path
        self.num_per_class = num_per_class
        self.class_names = class_names
        self.transforms = transformations
        self.channel_dim = channel_dim
        self.data = []
        self.labels = []

        if path:
            self.readImages()

        # Default transforms applied to the testing and validation data
        self.standard_transforms = transforms.Compose([
            transforms.Resize(img_size),
            transforms.ToTensor()
        ])

    def readImages(self):
        for id, class_name in self.class_names.items():
            print(f'Loading images from class: {id} : {class_name}')
            img_path = glob.glob(os.path.join(self.path, class_name, '*.jpg'))
            if self.num_per_class > 0:
                img_path = img_path[:self.num_per_class]
            self.labels.extend([id] * len(img_path))
            for filename in img_path:
                if self.channel_dim == rgb:
                    img = Image.open(filename).convert('RGB')
                else:
                    img = Image.open(filename).convert('L')
                self.data.append(img)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx]
        label = self.labels[idx]

        if self.transforms:
            img = self.transforms(img)  # Apply the custom transforms, including augmentations
        else:
            img = self.standard_transforms(img)  # Apply standard transforms if no custom ones are provided

        label = torch.tensor(label, dtype=torch.long)

        return img, label
    
    
# Paths to the data
train_path = "data/training/"
test_path = "data/testing/"
validation_path = "data/validation/"

class_names = [name[len(train_path):] for name in glob.glob(f'{train_path}*')]
class_names = dict(zip(range(len(class_names)), class_names))
class_names

# Define the data augmentation transformations
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(15),      # Randomly rotate the image by 15 degrees
    transforms.RandomResizedCrop(input_dim),  # Random crop followed by resizing
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Randomly change brightness, contrast, etc.
    transforms.ToTensor(),              # Convert the image to a tensor
])

# Create an instance of CustomDataset with augmentations
train_dataset = CustomDataset(
    img_size=input_dim,
    class_names=class_names,
    path=train_path,
    transformations=train_transform,
    channel_dim=channel_dim
)

train_dataloader = DataLoader(train_dataset, shuffle=True)

validation_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

validation_dataset = CustomDataset(
    img_size=input_dim,
    class_names=class_names,
    path=validation_path,
    transformations=validation_transform,
    channel_dim=channel_dim
)

validation_dataloader = DataLoader(validation_dataset, shuffle=True)

test_dataset = CustomDataset(
    img_size=input_dim,
    class_names=class_names,
    path=test_path,
    channel_dim=channel_dim
)

test_dataloader = DataLoader(test_dataset, shuffle=True)

Loading images from class: 0 : normal
Loading images from class: 1 : pneumonia
Loading images from class: 0 : normal
Loading images from class: 1 : pneumonia
Loading images from class: 0 : normal
Loading images from class: 1 : pneumonia


In [ ]:
class group_9(nn.Module):
    def __init__(self):
        super(group_9, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Correct the input size for fc1 based on the calculations
        self.fc1 = nn.Linear(64 * 125*125, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 2)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.maxpool1(x)
        x = x.view(x.size(0), -1)

        # Fully Connected Layers
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return self.softmax(x)

In [ ]:
model = group_9()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

model.to(device)

group_9(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1000000, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)

In [ ]:
from tqdm.notebook import tqdm
import time
pbar = None
def train(model, num_epochs: int = 3):
    for epoch in range(num_epochs):
        correct = 0
        total = 0
        model.train()

        pbar = tqdm(total=len(train_dataloader), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch", leave=True)

        for data, targets in train_dataloader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(data)

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            current_accuracy = correct / total * 100

            pbar.update(1)
            pbar.set_postfix(accuracy=f"{current_accuracy:.2f}%")

        pbar.close()
        tqdm.write(f"Epoch {epoch + 1}/{num_epochs}, Training accuracy: {current_accuracy:.2f}%")


        model.eval()
        correct_validation = 0
        total_validation = 0
        with torch.no_grad():
            for data, targets in validation_dataloader:
                data, targets = data.to(device), targets.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total_validation += targets.size(0)
                correct_validation += (predicted == targets).sum().item()

        validation_accuracy = 100 * correct_validation / total_validation
        print(f'Validation accuracy: {validation_accuracy}%')
        torch.save(model.state_dict(), f'drive/MyDrive/SDU_Data/models/model_weights_3_{epoch}.pth')

In [ ]:
def test(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in test_dataloader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

In [7]:
train(model=model, num_epochs=15)
test(model=model)

Epoch 1/15:   0%|          | 0/1740 [00:00<?, ?batch/s]

Epoch 1/15, Training accuracy: 53.05%
Validation accuracy: 50.0%


Epoch 2/15:   0%|          | 0/1740 [00:00<?, ?batch/s]

Epoch 2/15, Training accuracy: 58.97%
Validation accuracy: 90.9090909090909%


Epoch 3/15:   0%|          | 0/1740 [00:00<?, ?batch/s]

...

Epoch 13/15, Training accuracy: 94.02%
Validation accuracy: 95.45454545454545%


Epoch 14/15:   0%|          | 0/1740 [00:00<?, ?batch/s]

Epoch 14/15, Training accuracy: 94.20%
Validation accuracy: 92.72727272727273%


Epoch 15/15:   0%|          | 0/1740 [00:00<?, ?batch/s]

Epoch 15/15, Training accuracy: 95.29%
Validation accuracy: 95.45454545454545%
Test Accuracy: 92.73%


In [8]:
torch.save(model.state_dict(), 'group_9.pth')